In [1]:
import sys, os, math
sys.path.insert(0,os.path.expanduser('~/git/pyzx')) # git version
sys.path.insert(0,'/workspaces/pyzx')
import pyzx as zx
from pyzx import print_matrix
from pyzx.basicrules import *
from fractions import Fraction
import random
from itertools import combinations
from pyzx.fourier import *
from pyzx.symbolic import Poly, Term, Var

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [2]:
def bool_var(v):
    return Poly([(1, Term([(Var(v, True), 1)]))])

def const(i):
    return Poly([(i, Term([]))])

In [3]:
p = const(Fraction(3/2)) + bool_var("b1")
q = Fraction(3/2)
g = zx.Graph()
g.add_vertex(VertexType.Z, phase=p)
print(zx.drawing.phase_to_s(p, poly_with_pi=True))
zx.draw(g)

(3/2 + b1)π


In [4]:
g = zx.Graph()
qs = 3
ins  = [g.add_vertex(B, qubit=i, row=0) for i in range(qs)]
zs   = [g.add_vertex(Z, qubit=i, row=2) for i in range(qs)]
outs = [g.add_vertex(B, qubit=i, row=4) for i in range(qs)]

hb = g.add_vertex(VertexType.H_BOX, qubit=-1, row=3)
g.set_phase(hb, Fraction(1,4))
for i in range(qs):
    g.add_edge((ins[i], zs[i]))
    g.add_edge((zs[i], outs[i]))
    g.add_edge((hb, zs[i]))
zx.draw(g, labels=True)

In [5]:
g1 = g.copy()
print(check_fourier(g1, hb))
fourier(g1, hb)
zx.draw(g1)

True


In [6]:
g = zx.Graph()
qs = 3
ins  = [g.add_vertex(B, qubit=i, row=0) for i in range(qs)]
zs   = [g.add_vertex(Z, qubit=i, row=2) for i in range(qs)]
outs = [g.add_vertex(B, qubit=i, row=4) for i in range(qs)]

v1 = g.add_vertex(VertexType.Z, qubit=-2, row=3)
v2 = g.add_vertex(VertexType.X, qubit=-1, row=3)
g.add_edge((v1,v2))
g.set_phase(v1, Fraction(1,4))
for i in range(qs):
    g.add_edge((ins[i], zs[i]))
    g.add_edge((zs[i], outs[i]))
    g.add_edge((v2, zs[i]))
zx.draw(g, labels=True)

In [7]:
g1 = g.copy()
print(check_ifourier(g1, v1))
ifourier(g1, v1)
zx.draw(g1)

True


In [8]:
g = zx.Graph()
i1 = g.add_vertex(qubit=0, row=0)
i2 = g.add_vertex(qubit=2, row=0)
va = g.add_vertex(Z, phase=Fraction(1/4), qubit=1, row=0)
vb = g.add_vertex(X, qubit=1,row=1)
vc = g.add_vertex(Z, phase=1, qubit=0, row=2)
vd = g.add_vertex(Z, phase=0, qubit=2, row=2)
ve = g.add_vertex(X, qubit=0, row=4)
vf = g.add_vertex(X, phase=Fraction(1/2), qubit=2, row=4)
o1 = g.add_vertex(qubit=0, row=5)
o2 = g.add_vertex(qubit=2, row=5)
for e in [(i1, vc), (i2, vd), (va, vb), (vb, vc), (vb, vd), (vc, ve), (vc, vf), (vd, ve), (ve, o1), (vf, o2)]:
    g.add_edge(e)
zx.draw(g, labels=True)

In [9]:
o, c = zx.gflow.gflow(g.adjoint(), pauli=True, reverse=True, focus=True)
print(o)
print(c)

{2: 1, 4: 1, 5: 1, 7: 1, 3: 2, 6: 2}
{2: {3, 6}, 4: {4, 5, 7}, 5: {4, 5, 6, 7}, 7: {4, 5}, 3: {2}, 6: {2, 5}}


In [10]:
def unmap_qubit(p, width):
    y,z = p
    return y + z*width

def map_qubits(g, width):
    for v in g.vertices():
        q = g.qubit(v)
        z = math.floor(q/width)
        y = q - z*width
        g.set_qubit(v, y)
        g.set_vdata(v, 'z', z)

def random_nn_cnots(qubits, width, n):
    height = math.floor(qubits/width)
    c = zx.Circuit(qubits)
    g = 0
    for _ in range(n):
        q = (random.randint(0,height-1), random.randint(0,width-1))
        ns = [(q[0]-1, q[1]), (q[0]+1, q[1]), (q[0], q[1]-1), (q[0], q[1]+1)]
        q1 = random.choice([(y,z) for y,z in ns if y >= 0 and y < height and z >= 0 and z < width])
        c.add_gate('CNOT', unmap_qubit(q, width), unmap_qubit(q1, width))
    return c

In [11]:
qubits = 9
width = 3
gates = 20

random.seed(1337)
c = random_nn_cnots(qubits, width, gates)
print(c.gates)
g = c.to_graph()
map_qubits(g, width)

[CNOT(8,5), CNOT(8,7), CNOT(4,1), CNOT(4,5), CNOT(5,4), CNOT(7,8), CNOT(6,3), CNOT(4,1), CNOT(2,5), CNOT(2,5), CNOT(0,3), CNOT(8,5), CNOT(5,4), CNOT(2,1), CNOT(3,6), CNOT(2,1), CNOT(2,1), CNOT(3,4), CNOT(7,4), CNOT(1,4)]


In [12]:
zx.draw_3d(g)

In [13]:
def connect(g, vs, ws):
    if len(vs) == 0: return
    for i in range(len(vs)):
        for j in range(len(vs[0])):
            g.add_edge((vs[i][j], ws[i][j]))

def add_boundary(g, x, yrange, zrange):
    vs = [[g.add_vertex(B, row=x, qubit=y) for _ in zrange] for y in yrange]
    for y in yrange:
        for z in zrange:
            g.set_vdata(vs[y-yrange.start][z-zrange.start], 'z', z)
    return vs

def add_patch(g, ty, x, yrange, zrange, orientation='H'):
    ty_op = X if ty == Z else Z
    vs = [[g.add_vertex(ty, row=x, qubit=y) for z in zrange] for y in yrange]
    parity = 0 if ty == Z else 1
    for y in range(yrange.start, yrange.stop+1):
        for z in range(zrange.start, zrange.stop+1):
            if y in yrange and z in zrange: g.set_vdata(vs[y-yrange.start][z-zrange.start], 'z', z)
    
            if y == yrange.start: yshift = -0.3
            elif y == yrange.stop: yshift = -0.7
            else: yshift = -0.5

            if z == zrange.start: zshift = -0.3
            elif z == zrange.stop: zshift = -0.7
            else: zshift = -0.5
                
            if (orientation == 'H' and yshift != -0.5) or (orientation == 'V' and zshift != -0.5):
                continue
            
            if (y+z) % 2 == parity:
                w = g.add_vertex(ty_op, row=x, qubit=y+yshift)
                g.set_vdata(w, 'z', z+zshift)
                for (i,j) in ((y-1, z-1), (y, z-1), (y-1, z), (y, z)):
                    if i in yrange and j in zrange:
                        g.add_edge((w, vs[i-yrange.start][j-zrange.start]))
    return vs

In [14]:
g = zx.Graph()
sz = range(0,3)
x = 0
p1 = add_boundary(g, x, sz, sz); x += 2
for _ in range(3):
    p = add_patch(g, Z, x, sz, sz, 'H'); x += 1
    connect(g, p1, p)
    p1 = add_patch(g, X, x, sz, sz, 'V'); x += 2
    connect(g, p, p1)
p = add_boundary(g, x, sz, sz)
connect(g, p1, p)

zx.draw_3d(g)

In [15]:
g = zx.Graph()
d = 7
x = 0

p = add_boundary(g, x, range(0,d), range(0,2*d))

x += d

for _ in range(d):
    x += 0.7 * (d/(d+1))
    p1a = add_patch(g, Z, x, range(0,d), range(0,d), 'H')
    p1b = add_patch(g, Z, x, range(0,d), range(d,2*d), 'H')
    p1 = [q+r for q,r in zip(p1a,p1b)]
    connect(g, p, p1)
    p = p1
    
    x += 0.3 * (d/(d+1))
    p1a = add_patch(g, X, x, range(0,d), range(0,d), 'V')
    p1b = add_patch(g, X, x, range(0,d), range(d,2*d), 'V')
    p1 = [q+r for q,r in zip(p1a,p1b)]
    connect(g, p, p1)
    p = p1

x += d

for _ in range(d):
    x += 0.7 * (d/(d+1))
    p1 = add_patch(g, Z, x, range(0,d), range(0,2*d), 'H')
    connect(g, p, p1)
    p = p1
    
    x += 0.3 * (d/(d+1))
    p1 = add_patch(g, X, x, range(0,d), range(0,2*d), 'V')
    connect(g, p, p1)
    p = p1

x += d

p1 = add_boundary(g, x, range(0,d), range(0,2*d))
connect(g, p, p1)
p = p1

for v in g.vertices():
    z = g.vdata(v, 'z')
    if z == d-0.5:
        g.set_vdata(v, 'z', z+(d/2))
    elif z > d-0.5:
        g.set_vdata(v, 'z', z+d)

zx.draw_3d(g)